In [83]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

import numpy as np

In [84]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [85]:
mean_value = np.mean(x_train)
std_value = np.std(x_train)
x_train = (x_train - mean_value) / std_value
x_test = (x_test - mean_value) / std_value

In [86]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state = 1234)

In [87]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3),
                        activation='relu',
                        input_shape=(28, 28, 1),
                        padding = "same")
         )
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3),
                        activation='relu',
                        padding = "same")
         )
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3),
                        activation='relu',
                        padding = "same")
         )
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [88]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 7, 7, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 7, 7, 64)          36928     
                                                                 
 flatten_8 (Flatten)         (None, 3136)             

In [89]:
model.compile(optimizer = "adam",
             loss = "categorical_crossentropy",
             metrics = ["accuracy"],
             )

early_stopping = EarlyStopping(monitor = "val_accuracy", patience = 10, restore_best_weights = True)

In [94]:
model.fit(x_train, y_train,
                  batch_size = 4096,
                  epochs = 100,
                  validation_data=(x_val, y_val),
                  callbacks=[early_stopping],
         )

Epoch 1/100
12/12 [==============================] - 5s 292ms/step - loss: 0.0474 - accuracy: 0.9828 - val_loss: 0.2721 - val_accuracy: 0.9308
Epoch 2/100
12/12 [==============================] - 1s 96ms/step - loss: 0.0466 - accuracy: 0.9835 - val_loss: 0.2712 - val_accuracy: 0.9320
Epoch 3/100
12/12 [==============================] - 1s 99ms/step - loss: 0.0420 - accuracy: 0.9849 - val_loss: 0.2771 - val_accuracy: 0.9315
Epoch 4/100
12/12 [==============================] - 1s 100ms/step - loss: 0.0423 - accuracy: 0.9840 - val_loss: 0.2760 - val_accuracy: 0.9309
Epoch 5/100
12/12 [==============================] - 1s 110ms/step - loss: 0.0399 - accuracy: 0.9855 - val_loss: 0.2845 - val_accuracy: 0.9308
Epoch 6/100
12/12 [==============================] - 1s 104ms/step - loss: 0.0399 - accuracy: 0.9855 - val_loss: 0.2839 - val_accuracy: 0.9302
Epoch 7/100
12/12 [==============================] - 1s 104ms/step - loss: 0.0388 - accuracy: 0.9862 - val_loss: 0.2837 - val_accuracy: 0.9308
E

In [95]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'ACCURACY: {test_acc * 100:.2f}%')

313/313 [==============================] - 1s 3ms/step - loss: 0.3050 - accuracy: 0.9277
ACCURACY: 92.77%
